In [1]:
import sys
sys.path.append('../')
import numpy as np
import manavlib.io.xml_io as xml_io
import manavlib.io.movingai_io as mai_io
import manavlib.common.params as params
import manavlib.gen.tasks as agents
import manavlib.gen.maps as grid
import random

## Check Grid Map File

In [2]:
moving_ai_file = "../xml-examples/random-32-32-10.map"
map_file = "../xml-examples/map.xml"
h_init, w_init, occupancy_grid_init = mai_io.read_map_file(moving_ai_file)
cell_size_init = 1.0
xml_io.create_map_file(map_file, occupancy_grid_init, cell_size_init)

In [3]:
h, w, cell_size, occupancy_grid = xml_io.read_xml_map(map_file)
assert h == h_init
assert w == w_init
assert cell_size == cell_size_init
assert np.all(occupancy_grid == occupancy_grid_init)
print("OK")

OK


## Check Agents Files

In [4]:
agent_params_types = set()
queue = [params.BaseAgentParams]
while queue:
    parent = queue.pop()
    for child in parent.__subclasses__():
        if child not in agent_params_types:
            agent_params_types.add(child)
            queue.append(child)

In [5]:

for agent_type in agent_params_types:
    default_agent_init = agent_type()

    for key, value in default_agent_init.__dict__.items():
        value_type = type(value)
        default_agent_init.__dict__[key] = value_type(random.randint(0, 100))

    agents_file = f"../xml-examples/default_params_{agent_type.model_name}_task.xml"
    start_states_init, goal_states_init = agents.create_random_empty_instance(10, 10, 10, cell_size, empty_cells_around = False, discrete=issubclass(agent_type, params.BaseDiscreteAgentParams))
    xml_io.create_agents_file(agents_file, start_states_init, goal_states_init, default_agent_init)
    default_agent, start_states, goal_states, agent_params = xml_io.read_xml_agents(agents_file)

    assert np.all(start_states == start_states_init)
    assert np.all(goal_states == goal_states_init)

    for key, value in default_agent_init.__dict__.items():
        assert default_agent_init.__dict__[key] == default_agent.__dict__[key]

print("OK")

OK


In [6]:

agents_params_init = []
for agent_type in agent_params_types:
    
    if agent_type is params.BaseDiscreteAgentParams:
        continue
    
    agent_init = agent_type()
    for key, value in agent_init.__dict__.items():
        value_type = type(value)
        agent_init.__dict__[key] = value_type(random.randint(0, 100))
    agents_params_init.append(agent_init)
default_agent_init = agents_params_init[0]
agents_file = f"../xml-examples/all_custom_params_task.xml"
start_states_init, goal_states_init = agents.create_random_empty_instance(len(agents_params_init), 100, 100, cell_size, empty_cells_around = False, discrete=False)
xml_io.create_agents_file(agents_file, start_states_init, goal_states_init, default_agent_init, agents_params_init)
default_agent, start_states, goal_states, agent_params = xml_io.read_xml_agents(agents_file)

assert np.all(start_states == start_states_init)
assert np.all(goal_states == goal_states_init)

for key, value in default_agent_init.__dict__.items():
    assert default_agent_init.__dict__[key] == default_agent.__dict__[key]

for agent, agent_init in zip(agents_params_init, agents_params_init):
    for key, value in agent_init.__dict__.items():
        assert agent_init.__dict__[key] == agent.__dict__[key]

print("OK")

OK


## Check Config File

In [7]:
class TestAlgParams(params.BaseAlgParams):
    alg_name = "test_alg"

    def __init__(self):
        self.a_param = 0
        self.b_param = 0.0

    def __str__(self) -> str:
        return self.alg_name + ": " + str(self.__dict__)
    
    def __repr__(self) -> str:
        return self.alg_name + ": " + str(self.__dict__)
    
    
class NewExperimentParams(params.ExperimentParams):
    def __init__(self) -> None:
        super().__init__()
        self.some_param1 = 0
        self.some_param2 = 0.0
    
params.ExperimentParams = NewExperimentParams

In [8]:
config_file = "../xml-examples/config.xml"

exp_params_init = params.ExperimentParams()

print(params.ExperimentParams().__dict__)
for key, value in exp_params_init.__dict__.items():
    value_type = type(value)
    exp_params_init.__dict__[key] = value_type(random.randint(0, 100))

alg_params_init = TestAlgParams()
for key, value in alg_params_init.__dict__.items():
    value_type = type(value)
    alg_params_init.__dict__[key] = value_type(random.randint(0, 100))

xml_io.create_config_file(config_file, alg_params_init, exp_params_init)

{'timestep': 0.0, 'xy_goal_tolerance': 0.0, 'max_steps': 0, 'some_param1': 0, 'some_param2': 0.0}


In [9]:
exp_params, alg_params = xml_io.read_xml_config(config_file)

for key, value in exp_params_init.__dict__.items():
    assert exp_params_init.__dict__[key] == exp_params.__dict__[key]
    
for key, value in alg_params_init.__dict__.items():
    assert alg_params_init.__dict__[key] == alg_params.__dict__[key]

print("OK")

OK
